In [7]:
import sys, os
sys.path.append(os.path.abspath('../src/'))

In [8]:
import utils
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans

In [ ]:
train, _ = utils.read_current('../data/round_255/')
train['era'] = train.era.str.extract('(\d+|X)$', expand=False).str.zfill(4)

In [ ]:
features = [f for f in train.columns if f.startswith('feature')]
len(features)

In [ ]:
pca = PCA(n_components=20)
X = train[features]
pca.fit(X)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
features_pca = pca.transform(X)

In [52]:
df = pd.DataFrame(features_pca)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.349383,-1.441543,-1.784223,-0.069887,1.732141,-0.401515,-0.813313,0.614254,0.100901,0.042094,-0.157250,-1.048096,0.858867,-0.532508,0.424294,0.241794,0.265617,0.293103,1.377726,-0.692124
1,-0.695061,-2.199358,0.393310,-0.565360,-0.887672,1.098793,-0.506295,-1.586090,-0.415844,-0.124202,-0.190029,0.196546,-1.144449,-0.922997,1.263770,-0.445617,0.407662,-0.714031,0.421495,0.215914
2,0.907658,-0.358363,-0.935120,0.445909,-1.228140,-1.925874,-0.951461,-0.613938,-0.915901,-0.898679,-1.056740,-0.358205,-0.726674,0.023696,-0.662500,0.859255,-0.895374,0.891931,0.293295,-0.648075
3,1.598844,-1.013377,-1.560691,-2.089594,-0.163381,-0.797296,-0.744242,-0.010343,0.514958,0.074685,-0.169040,-0.537557,0.770027,0.497964,-0.073147,0.419653,-0.669951,1.530355,-0.495693,-0.643575
4,-0.088821,-1.741689,-0.763104,-0.403335,-0.009067,1.187753,-0.384644,0.871196,1.096362,-2.811147,0.144743,-0.729262,-0.537267,-0.420900,-0.929340,0.015676,0.124679,0.318496,0.746548,0.298531


In [53]:
def cr_transform(df):
    cr = df.drop(columns='era').corr(method='spearman').values.ravel()
    cr = np.delete(cr, np.where(cr == 1.0))
    return pd.Series(cr)

In [54]:
df['era'] = train.era
cr_df = df.groupby('era').apply(cr_transform)

In [55]:
clustering = KMeans(n_clusters=2).fit(cr_df)

In [56]:
clustering.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [57]:
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.metrics import make_scorer

from scipy.stats import spearmanr

In [58]:
from lightgbm import LGBMRegressor

In [59]:
train1 = train[train.era.isin(cr_df.reset_index().era[clustering.labels_ == 0])].copy()
train2 = train[train.era.isin(cr_df.reset_index().era[clustering.labels_ == 1])].copy()

In [60]:
train1.shape

(190406, 314)

In [61]:
param = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "l2",
    "num_iterations": 2000,
    "learning_rate": 0.006,
    "lambda_l1": 1.4,
    "lambda_l2": 1.0,
    "bagging_fraction": 0.55,
    "bagging_freq": 1,
    "num_leaves": 107,
    "max_depth": 15,
    "verbose": 0,
    "random_state": 0,
}

In [19]:
def spearman(y_true, y_pred, axis=0):
    """ Calculate Spearman correlation """
    return spearmanr(y_true, y_pred, axis=axis)[0]

In [20]:
cv = GroupKFold(n_splits=10)

model = LGBMRegressor(**param)

X = train1[features]
y = train1.target
groups = train1.era

In [21]:
n_scores = cross_val_score(model, X, y, groups=groups, scoring=make_scorer(spearman), cv=cv)

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031623 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035394 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038102 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038956 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.040755 seconds.
You can set `force_row_wi

In [22]:
n_scores

array([0.02412099, 0.03468153, 0.02445085, 0.03866977, 0.02638618,
       0.02744172, 0.00998833, 0.03904774, 0.01676604, 0.00533853])

In [23]:
cv = GroupKFold(n_splits=10)

model2 = LGBMRegressor(**param)

X = train2[features]
y = train2.target
groups = train2.era

In [24]:
n_scores2 = cross_val_score(model2, X, y, groups=groups, scoring=make_scorer(spearman), cv=cv)

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047493 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052201 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047715 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049277 seconds.
You can set `force_row_wi

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051241 seconds.
You can set `force_row_wi

In [25]:
n_scores2

array([0.04917194, 0.05016584, 0.04742811, 0.04757191, 0.04455742,
       0.05235023, 0.04769052, 0.06047016, 0.04392662, 0.02904833])

In [62]:
model1 = LGBMRegressor(**param)
model1.fit(train1[features], train1.target)

model2 = LGBMRegressor(**param)
model2.fit(train2[features], train2.target)

/usr/local/Caskroom/miniconda/base/envs/nmr/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] bagging_fraction is set=0.55, subsample=1.0 will be ignored. Current value: bagging_fraction=0.55
[LightGBM] [Warning] lambda_l1 is set=1.4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041996 seconds.
You can set `force_row_wi

LGBMRegressor(bagging_fraction=0.55, bagging_freq=1, lambda_l1=1.4,
              lambda_l2=1.0, learning_rate=0.006, max_depth=15, metric='l2',
              num_iterations=2000, num_leaves=107, objective='regression',
              random_state=0, verbose=0)

In [63]:
_, test = utils.read_current("../data/round_255/")
validation = test.query("data_type == 'validation'").copy()

In [70]:
validation.era.value_counts()

0210    5227
0211    5197
0212    5191
0209    5164
0205    5161
0204    5152
0206    5143
0203    5119
0208    5114
0202    5090
0201    5083
0199    5006
0207    4991
0198    4981
0197    4970
0200    4929
0131    4814
0132    4812
0130    4756
0129    4705
0125    4698
0127    4688
0126    4682
0122    4658
0128    4636
0124    4630
0123    4609
0121    4573
Name: era, dtype: Int64

In [87]:
X = validation[features]
validation['era'] = validation.era.str.extract('(\d+|X)$', expand=False).str.zfill(4)
features_pca = pca.transform(X)
df = pd.DataFrame(features_pca)
df['era'] = validation.era.values
cr_df = df.groupby('era').apply(cr_transform)
l = clustering.predict(cr_df)

In [88]:
val1 = validation[validation.era.isin(cr_df.reset_index().era[l == 0])].copy()
val2 = validation[validation.era.isin(cr_df.reset_index().era[l == 1])].copy()

In [89]:
val1["prediction"] = model1.predict(val1[features])
val2["prediction"] = model2.predict(val2[features])
val = pd.concat([val1, val2])

In [91]:
from sklearn.metrics import mean_absolute_error


def spearmanr(target, pred):
    return np.corrcoef(target, pred.rank(pct=True, method="first"))[0, 1]


def sharpe_ratio(corrs: pd.Series) -> np.float32:
    """
        Calculate the Sharpe ratio for Numerai by using grouped per-era data

        :param corrs: A Pandas Series containing the Spearman correlations for each era
        :return: A float denoting the Sharpe ratio of your predictions.
        """
    return corrs.mean() / corrs.std()


# https://parmarsuraj99.medium.com/evaluating-financial-machine-learning-models-on-numerai-3562da8fd90
def calculate_feature_exposure(df, feature_names, prediction_name="prediction") -> list:
    """
    Example:
    -----
    feature_exposure, max_feat_exposure, square_sum_feature_exposure = calculate_feature_exposure(df, feature_names)
    """

    exposures = []
    for feature_name in feature_names:
        exposures.append(spearmanr(df[feature_name], df[prediction_name]))

    max_feat_exposure = np.max(np.abs(exposures))
    square_sum_feature_exposure = np.sum([e ** 2 for e in exposures])
    feature_exposure = np.std(exposures)

    return [feature_exposure, max_feat_exposure, square_sum_feature_exposure]


# Calculating Max Drawdown
def max_drawdown(df, prediction_name="prediction", target_name="target"):
    scores_per_era = df.groupby("era").apply(
        lambda x: spearmanr(x[prediction_name], x[target_name])
    )

    rolling_max = (
        (scores_per_era + 1).cumprod().rolling(window=100, min_periods=1).max()
    )
    daily_value = (scores_per_era + 1).cumprod()
    max_drawdown = (rolling_max - daily_value).max()

    return max_drawdown


def evaluate(df: pd.DataFrame, features) -> tuple:
    """
        Evaluate and display relevant metrics for Numerai 

        :param df: A Pandas DataFrame containing the columns "era", "target" and a column for predictions
        :param pred_col: The column where the predictions are stored
        :return: A tuple of float containing the metrics
        """

    def _score(sub_df: pd.DataFrame) -> np.float32:
        """Calculates Spearman correlation"""
        return spearmanr(sub_df["target"], sub_df["prediction"])

    # Calculate metrics
    corrs = df.groupby("era").apply(_score)
    print(corrs)
    payout_raw = (corrs / 0.2).clip(-1, 1)
    spearman = round(corrs.mean(), 4)

    payout = round(payout_raw.mean(), 4)
    numerai_sharpe = round(sharpe_ratio(corrs), 4)
    mae = mean_absolute_error(df["target"], df["prediction"]).round(4)
    drawdown = max_drawdown(df)
    fe, max_fe, square_sum_fe = calculate_feature_exposure(df, features)

    # Display metrics
    print(f"Spearman Correlation: {spearman}")
    print(f"Average Payout: {payout}")
    print(f"Sharpe Ratio: {numerai_sharpe}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Max drawdown: {drawdown}")
    print(
        f"Feature exposure: {fe}, Max Feature Exposure: {max_fe}, Square Sum: {square_sum_fe}"
    )
    return spearman, payout, numerai_sharpe, mae


spearman, payout, numerai_sharpe, mae = evaluate(val, features)

era
0121    0.048250
0122   -0.006862
0123    0.045411
0124    0.047519
0125    0.023273
0126    0.034841
0127    0.006922
0128    0.067730
0129    0.011226
0130    0.045947
0131    0.025225
0132    0.095965
0197    0.049618
0198    0.014236
0199   -0.037852
0200   -0.001051
0201    0.007699
0202    0.031593
0203    0.004123
0204    0.028376
0205    0.013274
0206   -0.013460
0207    0.078939
0208    0.066030
0209    0.032521
0210   -0.047850
0211    0.002811
0212    0.035348
dtype: float64
Spearman Correlation: 0.0254
Average Payout: 0.1268
Sharpe Ratio: 0.7774
Mean Absolute Error (MAE): 0.1553
Max drawdown: 0.0923023015635347
Feature exposure: 0.07845025550458165, Max Feature Exposure: 0.35343006184475806, Square Sum: 1.9094136424122787
